Here I have developed the AI-powered Content Creator Agent using the Llama 3.1 405B model, known for generating highly coherent and relevant text. This model is accelerated by NVIDIA LLM NIM microservices, ensuring exceptional computational efficiency and scalability.

Key to this project is the integration of LangChain’s ChatNVIDIA—an open-source Python library contributed by NVIDIA,which facilitates seamless interaction with NVIDIA NIM. Through LangChain runnable chain expressions (LCEL), we consolidated these capabilities into a streamlined, reliable, and structured workflow.

The Content Creator Agent not only simplifies the process but also delivers personalized, high-quality social media content tailored to user needs. By leveraging cutting-edge AI technologies, this project revolutionizes content creation, making it scalable and accessible for a wide range of users.

In [13]:
!pip install langchain-nvidia-ai-endpoints

In [46]:
import getpass
import os

import getpass
import os

## API Key can be found by going to NVIDIA NGC -> AI Foundation Models -> (some model) -> Get API Code or similar.
## 10K free queries to any endpoint (which is a lot actually).

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    global nvapi_key
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

NVAPI Key (starts with nvapi-): ··········


#Content Creator Agent
This section demonstrates the construction of a Content Creator Agent for generating promotional messages tailored to specific guidelines. The agent utilizes the following technologies and workflows:

NVIDIA API Endpoints: Employing the NVIDIA API catalog preview endpoints for message generation. For NVIDIA AI Enterprise customers, these endpoints can also be run locally via NIM.
LangChain Integration: The agent leverages LangChain\u2019s ChatNVIDIA to interact seamlessly with NVIDIA's advanced AI models and ensure structured output.
Template Design: A predefined system prompt guides the AI in generating promotional content based on the provided product description. The output adheres to a specific format that includes a title, message, and relevant hashtags.
Structured Output: Using LangChain\u2019s StructuredOutput capabilities, the agent organizes responses into fields such as Title, Message, and Tags, ensuring reliable and formatted results.

In [103]:
# test run and see that you can genreate a respond successfully
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain import prompts, chat_models, hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import Optional, List

## construct the system prompt
prompt_template = """
### [INST]

You are an expert social media content creator.
Your task is to create a different promotion message with the following
Product Description :
------
Achieve your fitness goals with our versatile and durable multi-purpose gym bench. Designed for strength training, cardio, and flexibility workouts, it features adjustable settings for various exercises, anti-slip padding for safety, and a compact foldable design for easy storage. Perfect for beginners and fitness enthusiasts alike
------

The output promotion message MUST use the following format :

'''
Title: a powerful, short message that dipict what this product is about
Message: be creative for the promotion message, but make it short and ready for social media feeds.
Tags: the hash tag human will nomally use in social media
'''

Begin!

[/INST]
 """
prompt = PromptTemplate(
input_variables=['produce_desc'],
template=prompt_template,
)



## provide the product_desc
product_desc="Achieve your fitness goals with our versatile and durable multi-purpose gym bench. Designed for strength training, cardio, and flexibility workouts, it features adjustable settings for various exercises, anti-slip padding for safety, and a compact foldable design for easy storage. Perfect for beginners and fitness enthusiasts alike"

## structural output using LMFE
class StructureOutput(BaseModel):
    Title: str = Field(description="Title of the promotion message")
    Message : str = Field(description="The actual promption message")
    Tags: List[str] = Field(description="Hash tags for social media, usually starts with #")

llm_with_output_structure=ChatNVIDIA(model="meta/llama-3.1-405b-instruct").with_structured_output(StructureOutput)

## construct the content_creator agent
content_creator = ( prompt | llm_with_output_structure )
out=content_creator.invoke({"product_desc":product_desc})

In [104]:
out

StructureOutput(Title='UNLEASH YOUR FITNESS GOALS!', Message='Get ready to upgrade your home gym with our versatile multi-purpose gym bench! With adjustable settings, anti-slip padding, and a compact foldable design, you can take your strength training, cardio, and flexibility workouts to the NEXT LEVEL! Perfect for beginners and fitness enthusiasts alike!', Tags=['#FitnessMotivation', '#HomeGymEssentials', '#GymBenchGoals', '#WorkoutInspiration', '#FitnessEnthusiast'])

In [105]:
out.Title

'UNLEASH YOUR FITNESS GOALS!'

In [106]:
out.Message

'Get ready to upgrade your home gym with our versatile multi-purpose gym bench! With adjustable settings, anti-slip padding, and a compact foldable design, you can take your strength training, cardio, and flexibility workouts to the NEXT LEVEL! Perfect for beginners and fitness enthusiasts alike!'

In [107]:
out.Tags

['#FitnessMotivation',
 '#HomeGymEssentials',
 '#GymBenchGoals',
 '#WorkoutInspiration',
 '#FitnessEnthusiast']

#Digital Artist Agent:
This section showcases the Digital Artist Agent, designed to transform promotional text into visually engaging content for social media campaigns. The agent integrates cutting-edge technologies to produce high-quality visuals based on user-provided prompts:

*  Text-to-Image Model: Utilizes NVIDIA's sdXL-turbo text-to-image model for generating creative visuals tailored to promotional needs.
*  Prompt Optimization: Automatically rewrites input promotional text into image generation prompts using a language model, ensuring alignment with visual design requirements.
*  High-Quality Output: Generates visually appealing images ideal for social media campaigns, enhancing the promotional impact of your content.

Key Workflow:
1. Rewrite Input: The agent refines the provided promotional title or description into a prompt optimized for image generation.
2. Image Generation: Communicates with the NVIDIA sdXL-turbo API to generate and save the image locally.
3. Output Location: The generated image is saved as output.jpg for easy retrieval and use


In [109]:
import requests
import base64, io
from PIL import Image
import requests, json
def generate_image(prompt :str) -> str :
    """
    generate image from text
    Args:
        prompt: input text
    """
    ## re-writing the input promotion title in to appropriate image_gen prompt
    gen_prompt=llm_rewrite_to_image_prompts(prompt)
    print("start generating image with llm re-write prompt:", gen_prompt)
    invoke_url = "https://ai.api.nvidia.com/v1/genai/stabilityai/sdxl-turbo"

    headers = {
        "Authorization": f"Bearer {nvapi_key}",
        "Accept": "application/json",
    }

    payload = {
        "text_prompts": [{"text": gen_prompt}],
        "seed": 0,
        "sampler": "K_EULER_ANCESTRAL",
        "steps": 2
    }

    response = requests.post(invoke_url, headers=headers, json=payload)

    response.raise_for_status()
    response_body = response.json()
    ## load back to numpy array
    print(response_body['artifacts'][0].keys())
    imgdata = base64.b64decode(response_body["artifacts"][0]["base64"])
    filename = 'output.jpg'
    with open(filename, 'wb') as f:
        f.write(imgdata)
    im = Image.open(filename)
    img_location=f"the output of the generated image will be stored in this path : {filename}"
    return img_location

This section introduces a Python script that refines user queries into concise, image-generation prompts optimized for creative output. The rewriter ensures that the transformed prompt aligns with specific stylistic and structural requirements for text-to-image models.

In [125]:
# test run and see that you can genreate a respond successfully

def llm_rewrite_to_image_prompts(user_query):
    prompt = prompts.ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Summarize the following user query into a very short, one-sentence theme for image generation, MUST follow this format : A iconic, futuristic image of , no text, no amputation, no face, bright, vibrant",
            ),
            ("user", "{input}"),
        ]
    )
    model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
    chain = ( prompt    | model   | StrOutputParser() )
    out= chain.invoke({"input":user_query})
    #print(type(out))
    return out

In [110]:
out=generate_image("UNLEASH YOUR FITNESS GOALS!")
out

start generating image with llm re-write prompt: A iconic, futuristic image of a vibrant and bright fitness center with advanced equipment, no text, no amputation, no face.
dict_keys(['base64', 'finishReason', 'seed'])


'the output of the generated image will be stored in this path : output.jpg'

In [111]:
llm=ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
llm_with_img_gen_tool=llm.bind_tools([generate_image],tool_choice="generate_image")

In [112]:
out=llm_with_img_gen_tool.invoke("Unlock Your Fitness Potential")
out.tool_calls

[{'name': 'generate_image',
  'args': {'prompt': 'Unlock Your Fitness Potential'},
  'id': 'chatcmpl-tool-20f6ceb54e4f4a1fb0c702fa3fa48967',
  'type': 'tool_call'}]

In [113]:
def output_to_invoke_tools(out):
    tool_calls=out.tool_calls
    ## check there are indeed tool_calls in the output
    if len(tool_calls) > 0 :
        ## assert the args attribute exists
        if 'args' in tool_calls[0] :

            prompt=tool_calls[0]['args']['prompt']
            output=generate_image(prompt)
        else:
            print("### out.tool_calls", out.tool_calls[0].keys() )
            output="cannot find input prompt from llm output, please rerun again"
    else:
        print("------------" , out)
        print("### out.tool_calls", out.tool_calls )
        output="agent did not find generate_image tool, please check the tool binding is successful"
    return output

In [114]:
digital_artist = (
    llm_with_img_gen_tool
    | output_to_invoke_tools
)


In [115]:
digital_artist.invoke("Hit Gym get fit")

start generating image with llm re-write prompt: A iconic, futuristic image of a man lifting weights and getting fit in a bright, vibrant gym setting, without text, face, or amputation.
dict_keys(['base64', 'finishReason', 'seed'])


'the output of the generated image will be stored in this path : output.jpg'

In [68]:
!pip install langchain-community
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00


#Integrating Human-in-the-Loop: Decision-Maker Role
To ensure oversight and maintain high-quality outputs, human decision-makers are integrated into the workflow. This process allows for review, refinement, and final approval of the results generated by the agents, promoting collaboration between AI and human expertise.

Key Features:
*  Human-Centered Oversight: Places humans at the center of decision-making, allowing for review and iteration of both text and image outputs to meet deployment standards.
*  Agent Selection: Human decision-makers evaluate the task requirements and choose the appropriate agent—either the Content Creator Agent or the Digital Artist Agent—for each task.
*  Multiple Iterations: Enables refinement of outputs through human review, ensuring promotional messages and visuals are polished and aligned with expectations.
Agentic Cognitive Architecture: Utilizes LangGraph to orchestrate agent workflows, ensuring seamless communication and task assignment.


In [116]:
# Or you can directly instantiate the tool
from langchain_community.tools import HumanInputRun
from langchain.agents import AgentType, load_tools
from langchain.agents import AgentType, initialize_agent, load_tools


def get_human_input() -> str:
    """ Put human as decision maker, human will decide which agent is best for the task"""

    print("You have been given 2 agents. Please select exactly _ONE_ agent to help you with the task, enter 'y' to confirm your choice.")
    print("""Available agents are : \n
            1 ContentCreator  \n
            2 DigitalArtist \n
            Enter 1 or 2""")
    contents = []
    while True:
        try:
            line = input()
            if line=='1':
                tool="ContentCreator"
                line=tool

            elif line=='2':
                tool="DigitalArtist"
                line=tool

            else:
                pass

        except EOFError:
            break
        if line == "y":
            print(f"tool selected : {tool} ")
            break
        contents.append(line)

    return "\n".join(contents)


# You can modify the tool when loading

ask_human = HumanInputRun(input_func=get_human_input)

In [117]:
## first we define GraphState
from typing import Dict, TypedDict
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
import operator

from langchain_core.messages import BaseMessage
class State(TypedDict):
    # The input string
    input: str
    input_to_agent : str
    agent_choice : str
    agent_use_tool_respond : str

In [70]:
!pip install colorama

This section demonstrates how i used LangGraph to create nodes that represent steps or actions in an agent-driven workflow. These nodes enable sequential or parallel execution of tasks, providing a structured and flexible approach to managing complex processes.

In [118]:
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolInvocation
from colorama  import Fore,Style
# Define the functions needed
def human_assign_to_agent(state):
    # ensure using original prompt
    inputs = state["input"]
    input_to_agent = state["input_to_agent"]

    concatenate_str = Fore.BLUE+inputs+ ' : '+Fore.CYAN+input_to_agent + Fore.RESET
    print(concatenate_str)
    print("---"*10)

    agent_choice=ask_human.invoke(concatenate_str)
    print(Fore.CYAN+ "choosen_agent : " + agent_choice + Fore.RESET)
    return {"agent_choice": agent_choice }

def agent_execute_task(state):
    inputs= state["input"]
    input_to_agent = state["input_to_agent"]
    print(Fore.CYAN+input_to_agent + Fore.RESET)
    # choosen agent will execute the task
    choosen_agent = state['agent_choice']
    if choosen_agent=='ContentCreator':
        structured_respond=content_creator.invoke({"product_desc":input_to_agent})
        respond='\n'.join([structured_respond.Title,structured_respond.Message,''.join(structured_respond.Tags)])
    elif choosen_agent=="DigitalArtist":
        respond=digital_artist.invoke(input_to_agent)
    else:
        respond="please reselect the agent, there are only 2 agents available: 1.ContentCreator or 2.DigitalArtist"


    print(Fore.CYAN+ "agent_output: \n" + respond + Fore.RESET)

    return {"agent_use_tool_respond": respond}

In [119]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(State)

# Define the two nodes
workflow.add_node("start", human_assign_to_agent)
workflow.add_node("end", agent_execute_task)

# This means that this node is the first one called
workflow.set_entry_point("start")
workflow.add_edge("start", "end")
workflow.add_edge("end", END)
app = workflow.compile()

In [122]:
my_query="create a good promption message for social promotion events using the following inputs"
product_desc="Achieve your fitness goals with our versatile and durable multi-purpose gym bench. Designed for strength training, cardio, and flexibility workouts, it features adjustable settings for various exercises, anti-slip padding for safety, and a compact foldable design for easy storage. Perfect for beginners and fitness enthusiasts alike"
respond=app.invoke({"input":my_query, "input_to_agent":product_desc})

create a good promption message for social promotion events using the following inputs : Achieve your fitness goals with our versatile and durable multi-purpose gym bench. Designed for strength training, cardio, and flexibility workouts, it features adjustable settings for various exercises, anti-slip padding for safety, and a compact foldable design for easy storage. Perfect for beginners and fitness enthusiasts alike
------------------------------


create a good promption message for social promotion events using the following inputs : Achieve your fitness goals with our versatile and durable multi-purpose gym bench. Designed for strength training, cardio, and flexibility workouts, it features adjustable settings for various exercises, anti-slip padding for safety, and a compact foldable design for easy storage. Perfect for beginners and fitness enthusiasts alike
You have been given 2 agents. Please select exactly _ONE_ agent to help you with the task, enter 'y' to confirm your choi

In [123]:
prompt_for_image=respond['agent_use_tool_respond'].split('\n')[0].split(':')[-1].strip()
prompt_for_image

'UNLEASH YOUR FITNESS POTENTIAL'

In [124]:
input_query="generate an image for me from the below promotion message"
respond2=app.invoke({"input":input_query, "input_to_agent":prompt_for_image})

generate an image for me from the below promotion message : UNLEASH YOUR FITNESS POTENTIAL
------------------------------


generate an image for me from the below promotion message : UNLEASH YOUR FITNESS POTENTIAL
You have been given 2 agents. Please select exactly _ONE_ agent to help you with the task, enter 'y' to confirm your choice.
Available agents are : 

            1 ContentCreator  

            2 DigitalArtist 
          
            Enter 1 or 2
2
y
tool selected : DigitalArtist 
choosen_agent : DigitalArtist
UNLEASH YOUR FITNESS POTENTIAL
start generating image with llm re-write prompt: "A iconic, futuristic image of a vibrant, bright gym showcasing advanced fitness technology, with no text, faces, or amputations."
dict_keys(['base64', 'finishReason', 'seed'])
agent_output: 
the output of the generated image will be stored in this path : output.jpg


In [80]:
im = Image.open('output.jpg')
im.show()

In [84]:
title = respond['agent_use_tool_respond'].split('\n')[0].split(':')[-1].strip() if '\n' in respond['agent_use_tool_respond'] else respond['agent_use_tool_respond'].split(':')[-1].strip()
try:
    promotion_msg = respond['agent_use_tool_respond'].split('\n')[1].split(':')[-1].strip()
except IndexError:
    promotion_msg = "No promotion message available."  # Or any other default value
hash_tags = ['#' + s for s in respond['agent_use_tool_respond'].split('\n')[-1].split(':')[-1].split('#') if s != ""]


In [85]:
hash_tag_in_md=[]
for hash_tag in hash_tags:

    temp=f"""<span>{hash_tag}</span>"""
    hash_tag_in_md.append(temp)

hashtags_in_md= '<br>'+ ''.join(hash_tag_in_md) + '</br>'

In [86]:
from IPython.display import Markdown, display

import markdown
markdown_str = markdown.markdown(f'''
<img src="output.jpg" width=600 height=480 class=center/>


#### {title}

{promotion_msg}

{hashtags_in_md}

''')

def printmd(markdown_str):
    display(Markdown(markdown_str))
printmd(markdown_str)

<p><img src="output.jpg" width=600 height=480 class=center/></p>
<h4>output.jpg</h4>
<p>No promotion message available.</p>
<p><br><span># output.jpg</span></br></p>

In [43]:
# test run and see that you can genreate a respond successfully
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain import prompts, chat_models, hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

def llm_rewrite_to_image_prompts(user_query):
    prompt = prompts.ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Summarize the following user query into a very short, one-sentence theme for image generation, MUST follow this format : A iconic, futuristic image of , no text, no amputation, no face, bright, vibrant",
            ),
            ("user", "{input}"),
        ]
    )
    model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
    chain = ( prompt    | model   | StrOutputParser() )
    out= chain.invoke({"input":user_query})
    #print(type(out))
    return out

In [44]:
import requests
import base64, io
from PIL import Image
import requests, json
def generate_image(prompt :str) -> str :
    """
    generate image from text
    Args:
        prompt: input text
    """
    ## re-writing the input promotion title in to appropriate image_gen prompt
    gen_prompt=llm_rewrite_to_image_prompts(prompt)
    print("start generating image with llm re-write prompt:", gen_prompt)
    invoke_url = "https://ai.api.nvidia.com/v1/genai/stabilityai/sdxl-turbo"

    headers = {
        "Authorization": f"Bearer {a}",
        "Accept": "application/json",
    }

    payload = {
        "text_prompts": [{"text": gen_prompt}],
        "seed": 0,
        "sampler": "K_EULER_ANCESTRAL",
        "steps": 2
    }

    response = requests.post(invoke_url, headers=headers, json=payload)

    response.raise_for_status()
    response_body = response.json()
    ## load back to numpy array
    print(response_body['artifacts'][0].keys())
    imgdata = base64.b64decode(response_body["artifacts"][0]["base64"])
    filename = 'output.jpg'
    with open(filename, 'wb') as f:
        f.write(imgdata)
    im = Image.open(filename)
    img_location=f"the output of the generated image will be stored in this path : {filename}"
    return img_location

In [45]:
out=generate_image("NVIDIA NeMo is a powerful SDK for all your GenAI needs")
out

/usr/local/lib/python3.10/dist-packages/langchain_nvidia_ai_endpoints/_common.py:176: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


Exception: [401] Unauthorized
Authentication failed
Please check or regenerate your API key.

In [34]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain import prompts, chat_models, hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate


def llm_rewrite_to_image_prompts(user_query):
    prompt = prompts.ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Summarize the following user query into a very short, one-sentence theme for image generation, MUST follow this format : A iconic, futuristic image of , no text, no amputation, no face, bright, vibrant",
            ),
            ("user", "{input}"),
        ]
    )
    model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
    chain = ( prompt    | model   | StrOutputParser() )
    out= chain.invoke({"input":user_query})
    #print(type(out))
    return out

In [35]:
## bind image generation as tool into llama3.1-405b llm
llm=ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
llm_with_img_gen_tool=llm.bind_tools([generate_image],tool_choice="generate_image")
## use LCEL to construct Digital Artist Agent
digital_artist = (
    llm_with_img_gen_tool
    | output_to_invoke_tools
)

/usr/local/lib/python3.10/dist-packages/langchain_nvidia_ai_endpoints/_common.py:176: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


NameError: name 'output_to_invoke_tools' is not defined